### REQUIRED LIBRARY

In [1]:
%%capture
!git clone https://github.com/facebookresearch/audiocraft.git
%cd audiocraft
!pip install -e .
!pip install dora-search
!pip install numba

### MOUNT DRIVE TO IMPORT TRAINED MODEL THAT TRAINED EARLIER

In [2]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

### INSTALL AND IMPORT GRADIO FOR APPLICATION DEVELOPMENT

In [2]:
%%capture
!pip install gradio
import gradio as gr

### IMPORT REQUIRED PACKAGES TO USE TRAIN MODEL FOR INFERENCING 

In [3]:
from audiocraft.data.audio import audio_write
import IPython.display as ipd
from audiocraft.models import MusicGen
import numpy as np



WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cpu)
    Python  3.10.11 (you have 3.10.14)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


### LOADING TRAINED MODEL

In [4]:
# load your finetune
musicgen = MusicGen.get_pretrained('/content/drive/MyDrive/music_model')
musicgen.set_generation_params(duration=16)

c:\Users\APP\anaconda3\envs\music\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\APP\anaconda3\envs\music\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\APP\.cache\huggingface\hub\models--facebook--encodec_32khz. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admini

### GRADIO APP (MUSIC GENERATOR) WITH BUTTONS OF INSTUMENT AND MOOD TO GENERATE MUSIC

In [5]:
import gradio as gr
import librosa
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import librosa.display

# Assuming the generate_music and audio_write functions are properly defined elsewhere

def generate_music(user_input, duration):
    musicgen.set_generation_params(duration=duration)
    wavs = musicgen.generate([user_input])
    generated_audios = []

    for idx, one_wav in enumerate(wavs):
        file_path = f'{idx}.wav'
        audio_write(file_path, one_wav.cpu(), musicgen.sample_rate, strategy="loudness", loudness_compressor=True, add_suffix=False)
        generated_audios.append(file_path)

    return generated_audios[0]  # Return the first generated audio for simplicity

def append_tag(tag, user_input):
    if user_input:
        return f"{user_input}, {tag}"
    else:
        return tag

def append_tag1(tag, user_input):
    if user_input:
        return f"{user_input}, {tag}"
    else:
        return tag


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# MUSIC GENERATOR")

    with gr.Row():
        with gr.Column():
          user_input = gr.Textbox(label="Input Text", placeholder="Enter music style or description here", interactive=True, lines=9, max_lines=9, scale=1)

          gr.Markdown("# INSTRUMENTS")
          tags = ["accordion","acousticbassguitar","acousticguitar","bass","beat","bell","bongo","brass","cello","clarinet","classicalguitar",
    "computer","doublebass","drummachine","drums","electricguitar","electricpiano","flute","guitar","harmonica","harp","horn","keyboard",
    "oboe","orchestra","organ","pad","percussion","piano","pipeorgan","rhodes","sampler","saxophone","strings","synthesizer","trombone",
    "trumpet","viola","violin","voice"]
          with gr.Row():
           for tag in tags:
               tag_button = gr.Button(tag)
               tag_button.click(append_tag, inputs=[gr.State(tag), user_input], outputs=user_input)
        # Create tag buttons

          tags1 = ["action","adventure","advertising","background","ballad","calm","children","christmas","commercial","cool","corporate","dark",
                  "deep","documentary","drama","dramatic","dream","emotional","energetic","epic","fast","film","fun","funny","game","groovy","happy",
                  "heavy","holiday","hopeful","inspiring","love","meditative","melancholic","melodic","motivational","movie","nature","party",
                  "positive","powerful","relaxing","retro","romantic","sad","slow","soft","soundscape","space","sport","summer","trailer","travel",
                  "upbeat","uplifting"]


        with gr.Column():
          gr.Markdown("# MOOD THEME")
          with gr.Row():
           for tag in tags1:
               tag1_button = gr.Button(tag, variant="primary")
               tag1_button.click(append_tag1, inputs=[gr.State(tag), user_input], outputs=user_input)
    duration_slider = gr.Slider(label="Duration (seconds)", minimum=10, maximum=20, step=1, value=16)


    output_audio = gr.Audio(label="Generated Music")
    generate_button = gr.Button("Generate Music", scale=1, variant="primary")

    generate_button.click(generate_music, inputs=[user_input, duration_slider], outputs=[output_audio])

    with gr.Row():
        output_audio

demo.launch()


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.26.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.
